# Copernicus Marine Data Analysis Template

이 노트북은 코페르니쿠스 해양 데이터 분석을 위한 템플릿입니다.
필요에 따라 각 섹션을 수정하여 사용하세요.

## 목차
1. 환경 설정
2. 데이터 로딩
3. 데이터 탐색
4. 데이터 전처리
5. 분석
6. 시각화
7. 결과 저장

## 1. 환경 설정

In [ ]:
# 기본 라이브러리
import os
import sys
import warnings
from datetime import datetime, timedelta

# 데이터 처리
import numpy as np
import pandas as pd
import xarray as xr

# 시각화
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Copernicus 유틸리티 모듈
sys.path.append('..')
import copernicus_utils as cu

# 경고 무시
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("환경 설정 완료!")

## 2. 데이터 로딩

In [ ]:
# 분석 설정
config = {
    'data_path': '../data/your_data.nc',  # 데이터 파일 경로
    'variable': 'sst',  # 분석할 변수
    'region': {
        'lon_range': (120, 140),  # 경도 범위
        'lat_range': (30, 45)      # 위도 범위
    },
    'time_range': {
        'start': '2020-01-01',
        'end': '2023-12-31'
    },
    'output_dir': '../output/'  # 결과 저장 경로
}

# 출력 디렉토리 생성
os.makedirs(config['output_dir'], exist_ok=True)

In [ ]:
# 데이터 로딩
# ds = cu.load_dataset(config['data_path'])

# 샘플 데이터 생성 (실제 사용시 위 코드로 대체)
import numpy as np
import pandas as pd
import xarray as xr

# 시간, 위도, 경도 좌표 생성
time = pd.date_range('2020-01-01', '2023-12-31', freq='D')
lat = np.arange(30, 45.5, 0.5)
lon = np.arange(120, 140.5, 0.5)

# 샘플 SST 데이터 생성 (계절성 + 노이즈)
sst_data = np.zeros((len(time), len(lat), len(lon)))
for t in range(len(time)):
    seasonal = 20 + 10 * np.sin(2 * np.pi * t / 365.25)
    for i in range(len(lat)):
        for j in range(len(lon)):
            lat_effect = (45 - lat[i]) / 15 * 5
            sst_data[t, i, j] = seasonal + lat_effect + np.random.randn() * 2

# xarray Dataset 생성
ds = xr.Dataset(
    {
        'sst': (['time', 'latitude', 'longitude'], sst_data),
        'salinity': (['time', 'latitude', 'longitude'], 
                    35 + np.random.randn(len(time), len(lat), len(lon)) * 0.5)
    },
    coords={
        'time': time,
        'latitude': lat,
        'longitude': lon
    }
)

# 속성 추가
ds['sst'].attrs['units'] = '°C'
ds['sst'].attrs['long_name'] = 'Sea Surface Temperature'
ds['salinity'].attrs['units'] = 'PSU'
ds['salinity'].attrs['long_name'] = 'Sea Surface Salinity'

print("샘플 데이터 생성 완료!")

## 3. 데이터 탐색

In [ ]:
# 데이터셋 정보 확인
print("=" * 50)
print("데이터셋 정보:")
print("=" * 50)
print(ds)
print("\n")

# 변수 목록 확인
var_info = cu.list_variables(ds)
print("변수 정보:")
print(var_info)

In [ ]:
# 데이터 품질 검사
qc_result = cu.quality_check(
    ds, 
    config['variable'],
    valid_range=(-2, 40),  # SST 유효 범위
    min_coverage=0.8
)

print("\n품질 검사 결과:")
for key, value in qc_result.items():
    if key not in ['shape', 'long_gaps']:
        print(f"  {key}: {value}")

## 4. 데이터 전처리

In [ ]:
# 지역 추출
ds_region = cu.subset_region(
    ds,
    lon_range=config['region']['lon_range'],
    lat_range=config['region']['lat_range']
)

# 시간 추출
ds_subset = cu.subset_time(
    ds_region,
    start_date=config['time_range']['start'],
    end_date=config['time_range']['end']
)

print(f"\n전처리 후 데이터 크기: {dict(ds_subset.dims)}")

## 5. 분석

In [ ]:
# 5.1 공간 평균 시계열 생성
ts_mean = cu.create_timeseries(
    ds_subset,
    config['variable'],
    spatial_mean=True
)

print(f"시계열 길이: {len(ts_mean)}")
print(f"평균값: {ts_mean.mean():.2f}")
print(f"표준편차: {ts_mean.std():.2f}")

In [ ]:
# 5.2 이상치(Anomaly) 계산
anomaly = cu.calculate_anomaly(
    ds_subset,
    config['variable'],
    reference_period=(config['time_range']['start'], '2021-12-31')
)

print("이상치 계산 완료")

In [ ]:
# 5.3 트렌드 분석
trend = cu.calculate_trend(ts_mean, method='linear')

print("\n트렌드 분석 결과:")
print(f"  기울기: {trend['slope']:.4f}")
print(f"  R²: {trend['r_squared']:.3f}")
print(f"  p-value: {trend['p_value']:.4f}")
if 'annual_trend' in trend:
    print(f"  연간 트렌드: {trend['annual_trend']:.3f}°C/year")

In [ ]:
# 5.4 이동평균 적용
ts_smooth = cu.apply_moving_average(ts_mean, window=30, center=True)

print("30일 이동평균 적용 완료")

In [ ]:
# 5.5 극값 이벤트 탐지
extremes = cu.detect_extremes(
    ts_mean,
    threshold_type='percentile',
    threshold_value=95,
    duration=5
)

print(f"\n극값 이벤트: {len(extremes)}개")
if len(extremes) > 0:
    print("\n상위 3개 이벤트:")
    print(extremes.nlargest(3, 'max_value')[['start', 'end', 'duration', 'max_value']])

## 6. 시각화

In [ ]:
# 6.1 공간 분포 지도
fig, ax = cu.plot_map(
    ds_subset,
    config['variable'],
    time_idx='2023-07-01',
    figsize=(14, 8),
    cmap='RdYlBu_r',
    title=f"{config['variable'].upper()} Spatial Distribution - 2023-07-01"
)
plt.show()

In [ ]:
# 6.2 시계열 플롯
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 원본 시계열과 이동평균
ax1 = axes[0]
ax1.plot(ts_mean.index, ts_mean.values, label='Original', alpha=0.5, linewidth=0.5)
ax1.plot(ts_smooth.index, ts_smooth.values, label='30-day Moving Average', linewidth=2)
ax1.plot(trend['trend_line'].index, trend['trend_line'].values, 
         label=f'Linear Trend ({trend["slope"]:.4f}/day)', linestyle='--', linewidth=2)
ax1.set_ylabel(f'{config["variable"].upper()} (°C)')
ax1.set_title('Time Series Analysis')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# 이상치
ax2 = axes[1]
anomaly_ts = cu.create_timeseries(xr.Dataset({config['variable']: anomaly}), 
                                  config['variable'], spatial_mean=True)
ax2.fill_between(anomaly_ts.index, 0, anomaly_ts.values, 
                  where=(anomaly_ts.values > 0), color='red', alpha=0.5, label='Positive')
ax2.fill_between(anomaly_ts.index, 0, anomaly_ts.values, 
                  where=(anomaly_ts.values < 0), color='blue', alpha=0.5, label='Negative')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_ylabel('Anomaly (°C)')
ax2.set_xlabel('Date')
ax2.set_title('Temperature Anomaly')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 6.3 월별 평균 박스플롯
monthly_data = []
months = []

for month in range(1, 13):
    month_data = ts_mean[ts_mean.index.month == month]
    if len(month_data) > 0:
        monthly_data.append(month_data.values)
        months.append(month)

fig, ax = plt.subplots(figsize=(12, 6))
bp = ax.boxplot(monthly_data, positions=months, widths=0.6)
ax.set_xlabel('Month')
ax.set_ylabel(f'{config["variable"].upper()} (°C)')
ax.set_title('Monthly Distribution')
ax.set_xticks(months)
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.grid(True, alpha=0.3)
plt.show()

## 7. 결과 저장

In [ ]:
# 7.1 분석 결과 저장
results = {
    'config': config,
    'statistics': {
        'mean': float(ts_mean.mean()),
        'std': float(ts_mean.std()),
        'min': float(ts_mean.min()),
        'max': float(ts_mean.max())
    },
    'trend': {
        'slope': trend['slope'],
        'r_squared': trend['r_squared'],
        'p_value': trend['p_value']
    },
    'quality': qc_result,
    'extremes': len(extremes)
}

# JSON으로 저장
cu.save_config(results, os.path.join(config['output_dir'], 'analysis_results.json'))

In [ ]:
# 7.2 시계열 데이터 CSV 저장
ts_df = pd.DataFrame({
    'date': ts_mean.index,
    config['variable']: ts_mean.values,
    f'{config["variable"]}_smooth': ts_smooth.values,
    'trend': trend['trend_line'].values
})

cu.export_to_csv(
    ts_df,
    os.path.join(config['output_dir'], 'timeseries.csv')
)

In [ ]:
# 7.3 처리된 데이터셋 저장 (NetCDF)
output_nc = os.path.join(config['output_dir'], 'processed_data.nc')
ds_subset.to_netcdf(output_nc)
print(f"NetCDF 파일 저장: {output_nc}")

## 8. 분석 요약

### 주요 발견사항
- [여기에 주요 분석 결과를 요약하세요]
- [트렌드, 극값, 특이사항 등]

### 다음 단계
- [추가 분석이 필요한 부분]
- [다른 변수와의 상관관계 분석]
- [모델링 또는 예측]